In [2]:
import pandas as pd

import numpy as np

In [21]:
train = pd.read_csv("data/train.csv")

In [4]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [5]:
train.info()
# occyp_type 열 결측치 존재 ! 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [6]:
train.describe()
# FLAG_MOBIL :  핸드폰 소유 여부 (1=있다 0=없다) --> 제거
# child_num : 19명이 있는 집이 존재 
# DAYS_EMPLOYED	: 최대 값이 365243로, 평균이 59068인 것과 비교했을 때 이상치로 판단됨
# --> 확인해보니 DAYS_EMPLOYED 값이 36만 이상의 값을 가지면 모두 'occyp_type'이 nan임을 확인 --> 무직이라 판단

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


#### occyp_type 결측치 채우기 

In [8]:
train['occyp_type'].unique()
# nan 인 결측치 파악 필요

array([nan, 'Laborers', 'Managers', 'Sales staff',
       'High skill tech staff', 'Core staff', 'Drivers', 'Medicine staff',
       'Accountants', 'Realty agents', 'Security staff', 'Cleaning staff',
       'Private service staff', 'Cooking staff', 'Secretaries',
       'HR staff', 'IT staff', 'Low-skill Laborers',
       'Waiters/barmen staff'], dtype=object)

In [19]:
# DAYS_EMPLOYED (업무 시작일 : 수집 당시(0) 부터 역으로 셈, 즉 -1은 데이터 수집일 하루 전부터 일을 시작함, 양수 값은 고용되지 않은 상태를 의미 )

In [18]:
# DAYS_EMPLOYED 가 양수인 경우 = 고용되지 않은 상태  : 4438
train[train['DAYS_EMPLOYED'] > 0].index

Int64Index([   14,    18,    21,    24,    46,    48,    52,    58,    59,
               60,
            ...
            26389, 26401, 26402, 26406, 26427, 26431, 26432, 26439, 26441,
            26443],
           dtype='int64', length=4438)

In [20]:
# occyp_type(직업유형)이 값이 없는 경우 : 8171
train[train['occyp_type'].isnull()].index

Int64Index([    0,     8,    14,    18,    19,    20,    21,    23,    24,
               27,
            ...
            26431, 26432, 26433, 26435, 26437, 26439, 26441, 26443, 26449,
            26453],
           dtype='int64', length=8171)

In [23]:
# 직업유형이 결측치인 index와 DAYS_EMPLOYED가 고용되지 않은 상태인 index의 교집합 : 4438
len(set(train[train['occyp_type'].isnull()].index).intersection(set(train[train['DAYS_EMPLOYED'] > 0].index)))

4438

**결론**
- DAYS_EMPLOYED > 0인, 고용되지 않은 상태인 사람들은(4438명) 모두
- occyp_type 이 null 값이다. 
- occyp_type 의 결측치 중 4438개는 고용되지 않은 상태인 ``무직`` 상태이다 

##### occyp_type(직업 유형)에 ``무직`` 추가

In [6]:
train['occyp_type'].unique()

'''
nan, '노동자', '관리자', '영업 직원',
'고급 기술자', '핵심 직원', '운전사', '의약품 직원',
'회계사', '부동산 요원', '보안 직원', '청소 직원',
'개인 서비스 직원', '요리 직원', '비서',
'인사담당자', 'IT담당자', '저기술인력',
'웨이터/바텐 직원
'''

array([nan, 'Laborers', 'Managers', 'Sales staff',
       'High skill tech staff', 'Core staff', 'Drivers', 'Medicine staff',
       'Accountants', 'Realty agents', 'Security staff', 'Cleaning staff',
       'Private service staff', 'Cooking staff', 'Secretaries',
       'HR staff', 'IT staff', 'Low-skill Laborers',
       'Waiters/barmen staff'], dtype=object)

In [22]:
replace_idx = train[(train['occyp_type'].isnull()) & (train['DAYS_EMPLOYED'] > 0)]['occyp_type'].index

In [27]:
train.loc[replace_idx, 'occyp_type'] = train.loc[replace_idx, 'occyp_type'].fillna('Unemployed')

In [28]:
len(train[train['occyp_type'] == 'unemployed'])

4438

In [29]:
train['occyp_type'].unique()

array([nan, 'Laborers', 'Managers', 'Sales staff',
       'High skill tech staff', 'Core staff', 'Drivers', 'Medicine staff',
       'Accountants', 'unemployed', 'Realty agents', 'Security staff',
       'Cleaning staff', 'Private service staff', 'Cooking staff',
       'Secretaries', 'HR staff', 'IT staff', 'Low-skill Laborers',
       'Waiters/barmen staff'], dtype=object)

========================= 무직(unemployed) 추가 완료 =========================

In [30]:
# 처리 못한 결측치 : 3733
train['occyp_type'].isnull().sum()

3733

In [31]:
train.to_csv('data/train_null_pre1.csv', index=False)

In [32]:
len(train)

26457

In [33]:
len(train.dropna(axis=0))

22724

In [34]:
26457 - 3733

22724

In [35]:
train.dropna(axis=0, inplace=True)

In [36]:
train.to_csv("data/train_dropnull_pre2.csv", index=False)